In [5]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

### データ読み込み

In [12]:
# ユーザー特徴量を読み込み
user_features = np.load('user_features_standard.npy')
print(f"ユーザー特徴量の形状: {user_features.shape}")

# ユーザーマッピングを読み込み
with open('user_mapping_standard.json', 'r') as f:
    user_mapping = json.load(f)
print(f"ユーザーマッピング数: {len(user_mapping)}")

# BigQueryデータから性別情報を取得
df = pd.read_csv('bigquery_results.csv')

# 各ユーザーの性別を取得（最も頻度の高い性別を使用）
user_gender = df.groupby('common_id')['gender'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0])

print(f"性別分布:")
print(user_gender.value_counts())

# ユーザー特徴量と性別ラベルを対応付け
X = []
y = []
user_ids = []

for i, (user_id, features) in enumerate(zip(user_mapping.keys(), user_features)):
    if user_id in user_gender.index:
        X.append(features)
        y.append(user_gender[user_id])
        user_ids.append(user_id)

X = np.array(X)
y = np.array(y)

print(f"\n=== データセットの準備 ===")
print(f"特徴量の形状: {X.shape}")
print(f"ラベルの形状: {y.shape}")
print(f"性別分布（分類用）:")
unique, counts = np.unique(y, return_counts=True)
for gender, count in zip(unique, counts):
    print(f"  性別 {gender}: {count}人")


ユーザー特徴量の形状: (56, 20)
ユーザーマッピング数: 56
性別分布:
gender
1    34
2    22
Name: count, dtype: int64

=== データセットの準備 ===
特徴量の形状: (56, 20)
ラベルの形状: (56,)
性別分布（分類用）:
  性別 1: 34人
  性別 2: 22人


In [13]:
# データを訓練・テストに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"訓練データ: {X_train.shape[0]}サンプル")
print(f"テストデータ: {X_test.shape[0]}サンプル")

訓練データ: 39サンプル
テストデータ: 17サンプル


### 学習

In [14]:
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
}

results = {}

for name, clf in classifiers.items():
    print(f"\n--- {name} ---")
    
    # 訓練
    clf.fit(X_train, y_train)
    
    # 予測
    y_pred = clf.predict(X_test)
    
    # 評価
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"精度: {accuracy:.4f}")
    print(f"分類レポート:")
    print(classification_report(y_test, y_pred))



--- Logistic Regression ---
精度: 0.5882
分類レポート:
              precision    recall  f1-score   support

           1       0.59      1.00      0.74        10
           2       0.00      0.00      0.00         7

    accuracy                           0.59        17
   macro avg       0.29      0.50      0.37        17
weighted avg       0.35      0.59      0.44        17


--- Random Forest ---
精度: 0.7059
分類レポート:
              precision    recall  f1-score   support

           1       0.73      0.80      0.76        10
           2       0.67      0.57      0.62         7

    accuracy                           0.71        17
   macro avg       0.70      0.69      0.69        17
weighted avg       0.70      0.71      0.70        17

